# hw2

In [90]:
import numpy as np
import pandas as pd
import csv

## Discriminative Model

In [162]:
x=np.genfromtxt("X_train",delimiter=',',skip_header=1)
y=np.genfromtxt("Y_train",delimiter=',',skip_header=1)

In [92]:
mean=np.mean(x,axis=0)
std=np.std(x,axis=0)

boundry=[0,1,3,4,5]

for i in boundry:
    x[:,i]=(x[:,i]-mean[i])/std[i]

In [163]:
Min=np.min(x,axis=0)
Max=np.max(x,axis=0)

boundry=[0,1,3,4,5]

for i in boundry:
    x[:,i]=(x[:,i]-Min[i])/(Max[i]-Min[i])

In [164]:
def cross_entropy(y_pred,y_label):
    return -np.dot(y_label, np.log(y_pred))-np.dot((1-y_label), np.log(1-y_pred))

def sigmod(z):
    return np.clip(1/(1+np.exp(-z)),1e-6,1-1e-6)

In [165]:
def shuffle(x,y,size):
    batch=int(round(x.shape[0]*size))
    randomize=np.arange(batch)
    np.random.shuffle(randomize)
    return x[randomize],y[randomize]

def train(x,y,size):
    x=np.concatenate((np.ones((x.shape[0],1)),x),axis=1).astype(float)
    
    max_iteration=20000
    batch_size=64
    learning_rate=0.2
    lamda=0.001
    
    dim=x.shape[1]
    w=np.zeros((dim,))
    adagrad=np.zeros((dim,))
        
    step=0
    
    for epoch in range(max_iteration):
        if epoch%100==0:
            y_pred=sigmod(x@w)
            Y_pred=np.round(y_pred)
            print("Case:#",epoch)
            print("Accuracy:",np.sum(Y_pred==y)/len(y))
            print("Loss:",(cross_entropy(y_pred,y)+lamda*np.sum(np.square(w)))/len(y))
        
        x_train,y_train=shuffle(x,y,size)
        
        for idx in range(int(np.floor(x_train.shape[0]/batch_size))):
            X=x_train[idx*batch_size:(idx+1)*batch_size]
            Y=y_train[idx*batch_size:(idx+1)*batch_size]
            
            y_pred=sigmod(X@w)
            
            gradient=-np.mean(np.multiply((Y-y_pred).T,X.T),1)+lamda*w
#             adagrad+=gradient**2
            step+=1
#             w-=learning_rate/np.sqrt(adagrad+0.0001)*gradient
            w-=learning_rate/np.sqrt(step)*gradient
    
    return w

In [166]:
w=train(x,y,0.85)

Case:# 0
Accuracy: 0.7591904425539756
Loss: 0.6931471805599463
Case:# 100
Accuracy: 0.8272473204139922
Loss: 0.37898684573161834
Case:# 200
Accuracy: 0.829489266300175
Loss: 0.3727675232097831
Case:# 300
Accuracy: 0.8306870182119713
Loss: 0.36941460667869397
Case:# 400
Accuracy: 0.8317619237738398
Loss: 0.3671526567241605
Case:# 500
Accuracy: 0.8322225975860692
Loss: 0.36546376942750175
Case:# 600
Accuracy: 0.8326525598108167
Loss: 0.364126045645152
Case:# 700
Accuracy: 0.8332667915604558
Loss: 0.3630245082696941


KeyboardInterrupt: 

In [158]:
x_test=np.genfromtxt("X_test", delimiter=',', skip_header=1)
mean=np.mean(x_test,axis=0)
std=np.std(x_test,axis=0)

boundry=[0,1,3,4,5]

for i in boundry:
    x_test[:,i]=(x_test[:,i]-mean[i])/std[i]

In [161]:
x_test=np.concatenate((np.ones((x_test.shape[0],1)),x_test),axis=1).astype(float)
result=np.round(sigmod(x_test@w))
with open("ans.csv", 'w') as f:
        f.write('id,label\n')
        for i, v in  enumerate(result):
            f.write('%d,%d\n' %(i+1, v))

## Generative Model

In [168]:
import numpy as np
import sys
import csv  
from numpy.linalg import inv

In [169]:
class data_manager():
    def __init__(self):
        self.data = {}  
    
    def read(self,name,path):
        with open(path,newline = '') as csvfile:
            rows = np.array(list(csv.reader(csvfile))[1:] ,dtype = float)  
            if name == 'X_train':
                self.mean = np.mean(rows,axis = 0).reshape(1,-1)
                self.std = np.std(rows,axis = 0).reshape(1,-1)
                self.theta = np.ones((rows.shape[1] + 1,1),dtype = float) 
                for i in range(rows.shape[0]):
                    rows[i,:] = (rows[i,:] - self.mean) / self.std  

            elif name == 'X_test': 
                for i in range(rows.shape[0]):
                    rows[i,:] = (rows[i,:] - self.mean) / self.std 

            self.data[name] = rows  

    def find_theta(self):
        class_0_id = []
        class_1_id = []
        for i in range(self.data['Y_train'].shape[0]):
            if self.data['Y_train'][i][0] == 0:
                class_0_id.append(i)
            else:
                class_1_id.append(i)

        class_0 = self.data['X_train'][class_0_id]
        class_1 = self.data['X_train'][class_1_id] 

        mean_0 = np.mean(class_0,axis = 0)
        mean_1 = np.mean(class_1,axis = 0)  

        n = class_0.shape[1]
        cov_0 = np.zeros((n,n))
        cov_1 = np.zeros((n,n))
        
        for i in range(class_0.shape[0]):
            cov_0 += np.dot(np.transpose([class_0[i] - mean_0]), [(class_0[i] - mean_0)]) / class_0.shape[0]

        for i in range(class_1.shape[0]):
            cov_1 += np.dot(np.transpose([class_1[i] - mean_1]), [(class_1[i] - mean_1)]) / class_1.shape[0]

        cov = (cov_0*class_0.shape[0] + cov_1*class_1.shape[0]) / (class_0.shape[0] + class_1.shape[0])
 
        self.w = np.transpose(((mean_0 - mean_1)).dot(inv(cov)) )
        self.b =  (- 0.5)* (mean_0).dot(inv(cov)).dot(mean_0)\
            + 0.5 * (mean_1).dot(inv(cov)).dot(mean_1)\
            + np.log(float(class_0.shape[0]) / class_1.shape[0]) 

        result = self.func(self.data['X_train'])
        answer = self.predict(result)


    def func(self,x):
        arr = np.empty([x.shape[0],1],dtype=float)
        for i in range(x.shape[0]):
            z = x[i,:].dot(self.w) + self.b
            z *= (-1)
            arr[i][0] = 1 / (1 + np.exp(z))
        return np.clip(arr, 1e-8, 1-(1e-8))

    def predict(self,x):
        ans = np.ones([x.shape[0],1],dtype=int)
        for i in range(x.shape[0]):
            if x[i] > 0.5:
                ans[i] = 0; 
        return ans

    def write_file(self,path):
        result = self.func(self.data['X_test'])
        answer = self.predict(result)
        with open(path, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile) 
            writer.writerow(['id','label']) 
            for i in range(answer.shape[0]):
                writer.writerow([i+1,answer[i][0]])

In [170]:
dm = data_manager()
dm.read('X_train','X_train')
dm.read('Y_train','Y_train')
dm.read('X_test','X_test')
dm.find_theta()
dm.write_file('output.csv')